In [3]:
# setup qdrant ruby
require 'rubygems'
require 'qdrant'

client = Qdrant::Client.new(
  url: ENV['QDRANT_URL'],
  api_key: ENV['QDRANT_API_KEY']
)
puts "done setup client"

done setup client


# create collection

In [6]:
client.collections.create(
    collection_name: "collection1",     # required
    vectors: {
            "size": 4,
            "distance": "Dot"
        }                   # required
)

{"result"=>true, "status"=>"ok", "time"=>0.16361203}

# check collection

In [7]:
collection_name = "collection1"
client.collections.get(collection_name: collection_name)

{"result"=>{"status"=>"green", "optimizer_status"=>"ok", "vectors_count"=>0, "indexed_vectors_count"=>0, "points_count"=>0, "segments_count"=>2, "config"=>{"params"=>{"vectors"=>{"size"=>4, "distance"=>"Dot"}, "shard_number"=>1, "replication_factor"=>1, "write_consistency_factor"=>1, "on_disk_payload"=>true}, "hnsw_config"=>{"m"=>16, "ef_construct"=>100, "full_scan_threshold"=>10000, "max_indexing_threads"=>0, "on_disk"=>false}, "optimizer_config"=>{"deleted_threshold"=>0.2, "vacuum_min_vector_number"=>1000, "default_segment_number"=>0, "max_segment_size"=>nil, "memmap_threshold"=>nil, "indexing_threshold"=>20000, "flush_interval_sec"=>5, "max_optimization_threads"=>1}, "wal_config"=>{"wal_capacity_mb"=>32, "wal_segments_ahead"=>0}, "quantization_config"=>nil}, "payload_schema"=>{}}, "status"=>"ok", "time"=>1.5967e-05}

# Add points

In [8]:
client.points.upsert(
    collection_name: collection_name,
    points: [
          {"id": 1, "vector": [0.05, 0.61, 0.76, 0.74], "payload": {"city": "Berlin" }},
          {"id": 2, "vector": [0.19, 0.81, 0.75, 0.11], "payload": {"city": ["Berlin", "London"] }},
          {"id": 3, "vector": [0.36, 0.55, 0.47, 0.94], "payload": {"city": ["Berlin", "Moscow"] }},
          {"id": 4, "vector": [0.18, 0.01, 0.85, 0.80], "payload": {"city": ["London", "Moscow"] }},
          {"id": 5, "vector": [0.24, 0.18, 0.22, 0.44], "payload": {"count": [0] }},
          {"id": 6, "vector": [0.35, 0.08, 0.11, 0.44]}
        ]
)

{"result"=>{"operation_id"=>0, "status"=>"acknowledged"}, "status"=>"ok", "time"=>9.22e-05}

# Search

In [9]:
client.points.search(
    collection_name: collection_name,
    "vector": [0.2,0.1,0.9,0.7],
    "limit": 3
)

{"result"=>[{"id"=>4, "version"=>0, "score"=>1.362, "payload"=>nil, "vector"=>nil}, {"id"=>1, "version"=>0, "score"=>1.273, "payload"=>nil, "vector"=>nil}, {"id"=>3, "version"=>0, "score"=>1.208, "payload"=>nil, "vector"=>nil}], "status"=>"ok", "time"=>0.000169772}

# Search with filter

In [10]:
client.points.search(
    collection_name: collection_name,
    "filter": {
          "should": [
              {
                  "key": "city",
                  "match": {
                      "value": "London"
                  }
              }
          ]
      },
      "vector": [0.2, 0.1, 0.9, 0.7],
      "limit": 3
)

{"result"=>[{"id"=>4, "version"=>0, "score"=>1.362, "payload"=>nil, "vector"=>nil}, {"id"=>2, "version"=>0, "score"=>0.871, "payload"=>nil, "vector"=>nil}], "status"=>"ok", "time"=>0.000289354}